In [1]:
!pip install Cython numpy
!pip install git+https://github.com/lopuhin/python-adagram.git

  Cloning https://github.com/lopuhin/python-adagram.git to /tmp/pip-req-build-svszo4hd
  Running command git clone -q https://github.com/lopuhin/python-adagram.git /tmp/pip-req-build-svszo4hd
  Created wheel for adagram: filename=adagram-0.0.1-cp36-cp36m-linux_x86_64.whl size=464611 sha256=6bed2f4fba64a1807b0914ea3b112aaa8ca75c8ab05a49507311037485fef773
  Stored in directory: /tmp/pip-ephem-wheel-cache-y7m8x9qr/wheels/11/0f/46/f5df96670df8f7973b4c2311ffc9b02e435a7bd3207f992c4d
Successfully built adagram


In [2]:
!pip install pymorphy2

     |████████████████████████████████| 51kB 2.7MB/s 
     |████████████████████████████████| 7.1MB 7.2MB/s 


In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
import adagram
from lxml import html
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from pymorphy2 import MorphAnalyzer
from string import punctuation
import json, os
from collections import Counter
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))



def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split() if word and word not in stops]
    words = [word for word in words if word]

    return words

def normalize(text):
    
    words = tokenize(text)
    words = [morph.parse(word)[0].normal_form for word in words if word]

    return words

In [5]:
from google.colab import drive 
drive.mount('/gdrive')
%cd /gdrive


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
vm = adagram.VectorModel.load("/gdrive/My Drive/WSD_WSI/out.pkl")

Задание № 1. Протестировать адаграм в определении перефразирования

In [0]:
para = '/gdrive/My Drive/WSD_WSI/paraphraser/paraphrases.xml'

In [0]:
corpus_xml = html.fromstring(open(para, 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [0]:
data

,text_1,text_2,label
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0
2,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0
3,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1
4,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0
...,...,...,...
7222,Путин освободил от должности ряд генералов,Путин снял с должностей более 20 руководителей...,0
7223,Облака над Москвой в День Победы разгонят девя...,Путеводитель по Дню Победы: как провести 9 мая...,-1
7224,Любляна отпразднует День Победы вместе с Москвой,В Москве ограничат движение в связи с Днем Победы,-1
7225,Девять самолетов ВВС разгонят облака над Москв...,В Москве ограничат движение в связи с Днем Победы,-1


In [0]:
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

In [0]:
data.head(5)

,text_1,text_2,label,text_1_norm,text_2_norm
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,"[полицейский, разрешить, стрелять, поражение, ...","[полиция, мочь, разрешить, стрелять, хулиган, ..."
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,"[право, полицейский, проникновение, жилища, ре...","[правило, внесудебный, проникновение, полицейс..."
2,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0,"[президент, египет, ввести, чрезвычайный, поло...","[власть, египет, угрожать, ввести, страна, чре..."
3,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1,"[вернуться, сирия, россиянин, волновать, вопро...","[самолёт, мчс, вывезти, россиянин, разрушить, ..."
4,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0,"[москва, сирия, вернуться, 2, самолёт, мчс, ро...","[самолёт, мчс, вывезти, россиянин, разрушить, ..."


In [0]:
def get_words_in_context(words, n=3):
  neighbours = [] 
  for i in range(len(words)):
    if i-n<0:
      words_after = words[i+1:i+n+1]
      words_before = words[0:i]
      ws = words_before + words_after
      lst = list([words[i], ws])
      neighbours.append(lst)   

    else:
      words_after = words[i+1:i+n+1]
      words_before = words[i-n:i]
      ws = words_before + words_after
      lst = list([words[i], ws])
      neighbours.append(lst)

  return neighbours

In [7]:
words = [0,1,2,3,4,5,6,7,8,9]
get_words_in_context(words, n=3)

[[0, [1, 2, 3]],
 [1, [0, 2, 3, 4]],
 [2, [0, 1, 3, 4, 5]],
 [3, [0, 1, 2, 4, 5, 6]],
 [4, [1, 2, 3, 5, 6, 7]],
 [5, [2, 3, 4, 6, 7, 8]],
 [6, [3, 4, 5, 7, 8, 9]],
 [7, [4, 5, 6, 8, 9]],
 [8, [5, 6, 7, 9]],
 [9, [6, 7, 8]]]

In [0]:
def get_embedding_adagram(text, model, n, dim):
    
    word2context = get_words_in_context(text, n)
    
    
    vectors = np.zeros((len(word2context), dim))
    
    for i, (word, context) in enumerate(word2context):
        
        try:
            v = model.sense_vector(word, vm.disambiguate(word, context).argmax())
            vectors[i] = v
        
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector
    

In [0]:
dim=100
X_text_1 = np.zeros((len(data['text_1_norm']), dim))
X_text_2 = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1[i] = get_embedding_adagram(text, vm, n=3, dim=100)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2[i] = get_embedding_adagram(text, vm, n=3, dim=100)

In [0]:
X_text = np.concatenate([X_text_1, X_text_2], axis=1)

In [0]:
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import cross_val_score

In [0]:
y = data['label'].values

In [0]:
clf = svm.SVC(kernel='linear', C=1)

In [0]:
scores = cross_val_score(clf, X_text, y, cv=5, scoring='f1_macro')

In [0]:
scores

array([0.32923635, 0.33564626, 0.32806947, 0.28493003, 0.28372589])

In [0]:
scores.mean()

0.312321599532449

Задание 2. Реализовать алгоритм Леска и проверить его на реальном датасете

In [8]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
def lesk(word, sentence):
    bestsense = 0
    maxoverlap = 0
    synsets = wn.synsets(word)
    
    for i, syns in enumerate(synsets):
        senses = word_tokenize(syns.definition())
        overlap = len(set(senses).intersection(set(sentence)))
        if overlap > maxoverlap:
          maxoverlap = overlap
          bestsense = i
        
    return bestsense


In [10]:
lesk('day', 'some point or period in time'.split())

1

In [0]:
corpus_wsd = []
corpus = open('/gdrive/My Drive/WSD_WSI/corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

In [12]:
len(corpus_wsd)

49453

In [0]:
corpus_wsd_short = corpus_wsd[:1000]

In [0]:
wordnet_list = [] #список для значений ворднета
lesk_list = [] #список для значений леска

for i, sent in enumerate(corpus_wsd_short):
    if sent[0][0]:
      context = []

      for w in sent:
        if '%' in w[0]:
          context.append(w[1])
          wn_var = wn.lemma_from_key(w[0]).synset()
          wordnet_list.append(wn_var)
      
      words_in_context = get_words_in_context(context, n=3)
      
      for st in words_in_context:
        i = lesk(st[0], st[1])
        lesk_var = wn.synsets(st[0])[i]
        lesk_list.append(lesk_var)


In [0]:
d = {'wordnet': wordnet_list, 'lesk': lesk_list}
df = pd.DataFrame(data=d)

In [129]:
df.head(10)

,wordnet,lesk,match
0,Synset('be.v.01'),Synset('beryllium.n.01'),0.0
1,Synset('bigger.s.01'),Synset('bigger.s.01'),1.0
2,Synset('fancy.a.01'),Synset('fancy.n.02'),0.0
3,Synset('truly.r.01'),Synset('truly.r.01'),1.0
4,Synset('want.v.02'),Synset('need.n.01'),0.0
5,Synset('exist.v.01'),Synset('beryllium.n.01'),0.0
6,Synset('other.a.01'),Synset('other.a.01'),1.0
7,Synset('cheap.a.01'),Synset('cheap.a.01'),1.0
8,Synset('communication.n.01'),Synset('communication.n.01'),1.0
9,Synset('technique.n.01'),Synset('technique.n.01'),1.0


In [0]:
match = np.zeros(df.shape[0])

In [0]:
for i, syn in enumerate(df['wordnet'].values):
  if df['lesk'][i] == syn:
    match[i] = 1
  else:
    match[i] = 0

In [0]:
df['match'] = match

In [133]:
df.head(10)

,wordnet,lesk,match
0,Synset('be.v.01'),Synset('beryllium.n.01'),0.0
1,Synset('bigger.s.01'),Synset('bigger.s.01'),1.0
2,Synset('fancy.a.01'),Synset('fancy.n.02'),0.0
3,Synset('truly.r.01'),Synset('truly.r.01'),1.0
4,Synset('want.v.02'),Synset('need.n.01'),0.0
5,Synset('exist.v.01'),Synset('beryllium.n.01'),0.0
6,Synset('other.a.01'),Synset('other.a.01'),1.0
7,Synset('cheap.a.01'),Synset('cheap.a.01'),1.0
8,Synset('communication.n.01'),Synset('communication.n.01'),1.0
9,Synset('technique.n.01'),Synset('technique.n.01'),1.0


In [134]:
success = df[df['match'] == 1].shape[0]
print(success)

1430


In [135]:
all_vals = df['match'].shape[0]
print(all_vals)

2641


In [136]:
accuracy = success / all_vals
print(accuracy)

0.5414615675880349
